In [1]:
import polars as pl

In [2]:
!gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_03.parquet /home/eanegrin/datasets/

Copying file:///home/eanegrin/buckets/b1/datasets/competencia_03.parquet...
- [1 files][961.0 MiB/961.0 MiB]                                                
Operation completed over 1 objects/961.0 MiB.                                    


In [3]:
base_path = '/home/eanegrin/'
# base_path = 'C:/Eugenio/Maestria/DMEyF/'

dataset_path = base_path + 'datasets/'

dataset_file = 'competencia_03.parquet'

In [4]:
df = pl.read_parquet(dataset_path + dataset_file)

In [5]:
originales = df.select(pl.exclude(["numero_de_cliente", "foto_mes", "clase_ternaria"])).columns

In [6]:
# corrijo el tipo de estas 2:

df = df.with_columns([
    pl.col("tmobile_app").cast(pl.Int64),
    pl.col("cmobile_app_trx").cast(pl.Int64)
])

In [7]:
# lag 1
df = df.with_columns([
    pl.col(col)
    .sort_by(pl.col("foto_mes")) 
    .shift(1)
    .over("numero_de_cliente") 
    .alias(f"{col}_lag1")
    for col in originales
])

# lag 2
df = df.with_columns([
    pl.col(col)
    .sort_by(pl.col("foto_mes")) 
    .shift(2)
    .over("numero_de_cliente") 
    .alias(f"{col}_lag2")
    for col in originales
])

# delta 1
df = df.with_columns([
    (pl.col(col) - pl.col(f"{col}_lag1")).alias(f"{col}_delta1")
    for col in originales
])

# delta 2
df = df.with_columns([
    (pl.col(col) - pl.col(f"{col}_lag2")).alias(f"{col}_delta2")
    for col in originales
])

In [8]:
dataset_path

'/home/eanegrin/datasets/'

In [11]:
df.write_parquet(dataset_path + 'competencia_03_fe_v04.parquet')

In [12]:
!gsutil cp /home/eanegrin/datasets/competencia_03_fe_v04.parquet /home/eanegrin/buckets/b1/datasets/

Copying file:///home/eanegrin/datasets/competencia_03_fe_v04.parquet...
- [1 files][  2.7 GiB/  2.7 GiB]                                                
Operation completed over 1 objects/2.7 GiB.                                      
